In [10]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [11]:
#创建第一个交互式session
sess=tf.InteractiveSession()

#创建两个占位符，x为输入网络的图像，y_为输入网络的图像类别
x=tf.placeholder("float",shape=[None,784])
y_=tf.placeholder("float",shape=[None,10])

#权重初始化函数
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

#偏置初始化函数
def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#创建卷积op
#x是一个4维张量，shape为[batch,height,width,channels]
#卷积核移动步长为1。填充类型为SAME,可以不丢弃任何像素点
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")

#创建池化op
#采用最大池化，也就是窗口的最大值作为结果
# x是一个4维向量，shape为[batch,height,width,channels]
#ksize表示pool窗口大小为2*2，也就是高2，宽2
#strides表示在height和weight维度上的步长都为2
def max_pool_2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                          strides=[1,2,2,1], padding="SAME")

#第一层-卷积层
#初始化W为[5,5,1,32]的张量，表示卷积核大小为5*5，第一层网络的输入和输出神经元个数分别为1和32
W_conv1=weight_variable([5,5,1,32])
#初始化b为[32],即输出大小
b_conv1=bias_variable([32])

#把输入x(二维张量,shape为[batch, 784])变成4d的x_image，x_image的shape应该是[batch,28,28,1]
#-1表示自动推测这个维度的size
x_image=tf.reshape(x,[-1,28,28,1])

#把x_image和权重进行卷积，加上偏置项，然后应用ReLU激活函数，最后进行max_pooling
#h_pool1的输出即为第一层网络输出，shape为[batch,14,14,1]
h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1 = max_pool_2(h_conv1)

In [12]:
#第2层，卷积层
#卷积核大小依然是5*5，这层的输入和输出神经元个数为32和64
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = weight_variable([64])

#h_pool2即为第二层网络输出，shape为[batch,7,7,1]
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2(h_conv2)

In [14]:
#第三层 全连接层
W_fc1=weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])

#计算前需要把第二层的输出reshape成[batch,7*7*64]的张量
h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

#Dropout层
#为了减少过拟合，在输出层前加入dropout
keep_prob=tf.placeholder("float")
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

#输出层 
#最后，添加一个softax层
#可以理解为另一个全连接层，只不过输出时使用softmax层将网络输出值转换成了概率
W_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)
#预测值与真实值之间的交叉熵
cross_entropy=-tf.reduce_sum(y_*tf.log(y_conv))

#train_op,使用Adam优化器来做梯度下降。学习率为0.0001
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_predict=tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))

accuracy=tf.reduce_mean(tf.cast(correct_predict,"float"))

#初始化变量
sess.run(tf.global_variables_initializer())

#开始训练模型，循环20000次，每次随机从训练集中抓取50幅图像
for i in range(20000):
    batch=mnist.train.next_batch(50)
    if i%100==0:
        #每100次输出一次日志
        train_accuracy=accuracy.eval(feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
        print( "step %d, training accuracy %f" % (i, train_accuracy))
    
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})

step 0, training accuracy 0.060000
step 100, training accuracy 0.900000
step 200, training accuracy 0.860000
step 300, training accuracy 0.920000
step 400, training accuracy 0.960000
step 500, training accuracy 0.940000
step 600, training accuracy 0.940000
step 700, training accuracy 0.980000
step 800, training accuracy 0.880000
step 900, training accuracy 0.940000
step 1000, training accuracy 0.960000
step 1100, training accuracy 0.960000
step 1200, training accuracy 1.000000
step 1300, training accuracy 1.000000
step 1400, training accuracy 1.000000
step 1500, training accuracy 1.000000
step 1600, training accuracy 0.980000
step 1700, training accuracy 0.980000
step 1800, training accuracy 0.960000
step 1900, training accuracy 1.000000
step 2000, training accuracy 0.980000
step 2100, training accuracy 0.960000
step 2200, training accuracy 1.000000
step 2300, training accuracy 1.000000
step 2400, training accuracy 0.980000
step 2500, training accuracy 0.980000
step 2600, training accu

In [18]:
#保存权重
saver=tf.train.Saver()
save_path=saver.save(sess,"D:/record/LYJ.ckpt")
print("Model saved in file:",save_path)

#加载权重
saver=tf.train.Saver()
saver.restore(sess,"D:/record/LYJ.ckpt")
print("Model restored.")

Model saved in file: D:/record/LYJ.ckpt
INFO:tensorflow:Restoring parameters from D:/record/LYJ.ckpt
Model restored.


In [21]:
batch2=mnist.test.next_batch(1)
b=h_fc1.eval(feed_dict={x:batch2[0],y_:batch2[1],keep_prob:1.0})
print(batch2[1])
#h_fc1是全连接层的输出，eval()把特征tensor转成数组向量

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [36]:
#将测试集中所有图像都导入网络，提取特征
count=0 #相关总数
count1=0 #检索到相关的
count2=0 #所有检索出来的

for i in range(1):
    batch1=mnist.test.next_batch(10000)
    a=h_fc1.eval(feed_dict={x:batch1[0],y_:batch1[1],keep_prob:1.0})
    B=tf.argmax(batch1[1],1).eval()
    for j in range(10000):
        if B[j]==7: #检索的图像为数字7
            count+=1
    a1=tf.reshape(batch1[0],[10000,28,28]).eval()
    a2=255*a1
    a3=a2.astype(np.uint64)
    
    for k in range(10000):
        dist=np.linalg.norm(a[k]-b) #使用欧式距离
        if dist<=25: #阈值25
            count2+=1
            if B[k]==7:
                count1+=1

R=count1/count #查全率
P=count1/count2 #查准率

In [37]:
print(P)

0.13804828289697382


In [38]:
print(R)

0.7898832684824902
